
# GSOD Weather Data — SQL Overview

**Dataset:** `bigquery-public-data.noaa_gsod`  
**Primary tables:**
- `gsod*` (one table per year, e.g., `gsod2019`, `gsod2020`, …) — daily observations
- `stations` — station metadata (name, country, state, lat, lon, elevation, active period)

This notebook gives you structured prompts and blank cells to practice SQL in BigQuery.



## Why this dataset?
It’s simple but realistic, supports **joins**, and includes **data-quality issues** to clean:

- Date is split across `year`, `mo`, `da` (often typed as STRING in some yearly tables).  
- Missing values are represented by **sentinel numbers**, e.g. `9999.9`, `99.99`, `999.9`, `999.9` (varies by column).  
- Join keys: `stn` (USAF) and `wban` identify a station and can be joined to `stations` (use both keys when available).



## Getting started (choose one)
You can run these in:
1. **BigQuery Console** (UI): paste each SQL under the right section.
2. **Colab/Jupyter with BigQuery magic** (optional):
   - Install & auth (Colab):  
     ```python
     from google.colab import auth
     auth.authenticate_user()
     ```
   - Use cell magic:
     ```sql
     %%bigquery --project=project_id
     -- SQL here
     ```
**Note:** You may use a BigQuery notebook where you can use SQL cells, which eliminates the need of the bigquery magic cell.


---
## A) Basic exploration (work on a single year, e.g., `gsod2019`)
**Goal:** get familiar with the raw table.
### 1. Preview 5–10 rows of `bigquery-public-data.noaa_gsod.gsod2019`.

In [ ]:
%%bigquery --project=ba-770
SELECT * FROM `bigquery-public-data.noaa_gsod.gsod2019` LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,stn,wban,year,mo,da,temp,count_temp,dewp,count_dewp,slp,...,flag_min,prcp,flag_prcp,sndp,fog,rain_drizzle,snow_ice_pellets,hail,thunder,tornado_funnel_cloud
0,010014,99999,2019,08,31,66.6,4,57.2,4,9999.9,...,*,0.00,I,999.9,0,0,0,0,0,0
1,010350,99999,2019,03,03,8.8,4,9999.9,0,9999.9,...,None,0.00,I,999.9,0,0,0,0,0,0
2,010350,99999,2019,03,01,18.5,4,9999.9,0,9999.9,...,None,0.00,I,72.4,0,0,0,0,0,0
3,010430,99999,2019,11,27,30.3,4,9999.9,0,9999.9,...,*,0.00,I,999.9,0,0,0,0,0,0
4,010430,99999,2019,11,12,22.7,4,9999.9,0,9999.9,...,None,0.00,I,999.9,0,0,0,0,0,0
5,010430,99999,2019,12,22,26.6,4,9999.9,0,9999.9,...,None,0.00,I,999.9,0,0,0,0,0,0
6,010430,99999,2019,11,11,26.2,4,9999.9,0,9999.9,...,None,0.00,I,999.9,0,0,0,0,0,0
7,010430,99999,2019,11,18,27.6,4,9999.9,0,9999.9,...,None,0.00,I,999.9,0,0,0,0,0,0
8,010470,99999,2019,11,23,28.2,4,26.7,4,1021.9,...,None,0.00,G,10.2,0,0,0,0,0,0
9,011170,99999,2019,03,13,32.5,4,20.7,4,9999.9,...,*,0.01,E,999.9,0,0,0,0,0,0


### 2. List all columns and confirm data types.

### 3. Count total rows and number of distinct stations (`stn`, `wban`) for 2019.


- **`stn`** → Station number (a global identifier).  
- **`wban`** → Weather Bureau Air Force Navy number (mainly U.S. identifier).

Neither is unique on its own. Some stations share the same `stn` with different `wban` values, and others have no `wban`.
To ensure a unique station key across all years and countries, we combine them:

```sql
CONCAT(stn, '-', wban) AS station_id


### 4. Find the top 10 stations by number of reported days in 2019.


---
## B) Construct a proper DATE

### 1. build a `date` column as a true `DATE` from `year`, `mo`, `da`.
Using `DATE(CAST(year AS INT64), CAST(mo AS INT64), CAST(da AS INT64)) AS date)`, select a few rows to verify.


### 2. Create a **temporary cleaned subquery** (via `WITH`) that
* replaces `year`, `mo`, and `da` with `date`
* replaces `stn` and `wban` with our new `station_id`

## C) Handle missing values (sentinel codes)

### C.1 For columns like `dewp`, `slp`, `stp`, `visib`, `wdsp`, `mxpsd`, `gust`, `max`, `min`, `prcp`, find the sentinel value reported in the column description.

As noted in the notebook's introduction, common sentinel codes are `9999.9`, `99.99`, `999.9`, `999.9` (which varies by column). The best way to do this is to check the [original documentation](https://www.ncei.noaa.gov/pub/data/gsod/readme.txt?utm_source=chatgpt.com) and then verify.

*   `dewp`, `slp`, `max`, `min`: `9999.9`
*   `sndp`, `stp`, `wdsp`, `mxpsd`, `gust`, `visib`: `999.9`
*   `prcp`: `99.99`

### C.2 Count how many rows hit each sentinel value in 2019 (one query per column or a union).
Tip: Use the function `COUNTIF()`

### C.3 In a `WITH` subquery, `NULLIF` those sentinels. Let's also include the `station_id` and `date` columns we created earlier. Make sure to include all the other columns that we didn't treat so we don't lose any of the variables.

Additionally, `wdsp` and `mxpsd` were initially stored as `STRING` types, which is problematic for numerical analysis. Let's fix this by casting them to `FLOAT64` and then applying `NULLIF`:

```sql
NULLIF(CAST(wdsp AS FLOAT64), "999.9") AS wdsp,
NULLIF(CAST(mxpsd AS FLOAT64), "999.9") AS mxpsd,
```

Note: `NULLIF` helps you replace a specific value with **NULL**.

### C.4 Verify by re-counting NULLs after cleaning.

---

## D) Join with `stations` metadata

### D.1 Preview a few rows of `bigquery-public-data.noaa_gsod.stations`.

### D.2 Join your cleaned 2019 observations to `stations` on `station_id` where possible, and select relevant columns and in this order:

```date, station_id, name, country, state, lat, lon, elev, temp, dewp, max, min, prcp, wdsp```



### D.4 Count how many 2019 rows have **no match** in `stations` (left join) — report the count.

---

## E) Create a partitioned **clean** table

### E.1 Create a partitioned table `temp_dataset.gsod_2019_clean` with cleaned data.

### E.2 Confirm row count after table creation.

Observation: We lose 10697 points after the join. These are weather reports for stations that we don't have name or other pieces of information in the stations table.

---

## F) Intro summaries

### F.1 Average `temp` by month for 2019 (ignore NULLs).

### F.2 Number of rainy days per station (`prcp > 0`) in 2019; show top 10 stations.

### F.3 By country: average `max` and `min` temperature for 2019; keep only countries with at least 200 station-days (`HAVING`).

---

## G) Quality checks

### G.1 Show min/max of `temp`, `max`, `min`, `prcp` after cleaning.

### G.2 Count rows where `max < min` (should be rare or zero).

### G.3 Flag any station with `lat` or `lon` outside realistic bounds (after join).

---

## H) Bonus question

### H.1 Create a simple **UDF** (BigQuery SQL) that classifies a day as `'freezing'`, `'cool'`, `'warm'`, `'hot'` based on `temp`.